# Korea is refraining from moving because of COVID.
## The corona virus is getting worse, and many countries around the world are ordering bans to move. 

- Korea did not issue a ban on movement, but it is campaigning for <font size=3 color="red">"Social Distance Setting."</font>
- Let's find out where and how much the exercise was done through floating population data and patient data.



- 한국은 이동 금지 명령을 내리지는 않았습니다. 하지만 사회적 거리 두기 운동이 자율적으로 행해지고 있습니다. 
- 사회적 거리두기가 어떻게 이루어지고 있는지, 유동인구 데이터와 환자 데이터를 통해서 확인하고자 합니다. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import datetime
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Floating population data load and preprocessing 
- Floating population data is still available until February in Seoul
- Make a column related to time for visualization.


In [ ]:
fp = pd.read_csv('../input/coronavirusdataset/SeoulFloating.csv')

fp['dayofweek'] = fp['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").weekday()).astype('str')
fp['month'] = fp['date'].apply(lambda x: x[6:7]) 
fp['new_date'] = fp['month'].astype('str') + '/' + fp['date'].apply(lambda x: x[8:])
fp = fp.sort_values(['date', 'hour', 'birth_year', 'sex'])  ## this data is not sorted.
fp.reset_index(drop= True, inplace = True)


In [ ]:
fp.head()

## Time data load and preprocessing 
- This is the trend data of ***the number of confirmed persons, the number of negative judgments, and the number of covid testers over time.***
- All data are cumulative.
- Also make a same column related to time for visualization.


In [ ]:
time = pd.read_csv('/kaggle/input/coronavirusdataset/Time.csv')
time['month'] = time['date'].apply(lambda x: x[6:7]) 
time['new_date'] = time['month'].astype('str') + '/' + time['date'].apply(lambda x: x[8:])
time['dayofweek'] = time['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").weekday()).astype('str')


In [ ]:
time.head()

# 1. Total Correlation plot(each city)
### I will plot the correlation between the floating population and number of confirmers, and the number of covid testers by day of the week.

- <font size=3 color="red">A negative correlation </font> : people are not moving as the corona situation becomes serious
- <font size=3 color="blue"> a positive correlation </font> : more and more people are coming to this area.

- Seoul City has **25 gu**, which are subdivided into **424 dongs.**
- I will see **8 representative gu** among these gus. 8 gu are divided into  <font size=3 color="blue">4 residential areas</font> and  <font size=3 color="red">4 industrial areas.</font>


- 유동인구와 누적 감염자 수, 음성 판정 수, 코로나 검사자 수 사이의 요일별 상관관계를 살펴볼 것입니다. 
- 값이 음수 : COVID가 심해짐에 따라 유동인구가 감소한 것입니다. 
- 값이 양수 : COVID가 심해짐에 따라 유동인구가 증가한 것입니다. 
- 서울의 대표적인 8개의 구를 가져왔고, 그중 4개는 산업 지역, 4개는 주거 지역입니다.

In [ ]:
def plot_correlatiin(city, time, fp):
    holiday = ['1/01', '1/21','1/22','1/23','1/24','1/25','1/26','1/27','1/28','1/29','1/30']
    ## holiday is excluded.
    dayofweek_map = {0:'Monday', 1:'Tuesday', 2: 'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

    fig, ax = plt.subplots(3, 2, figsize = (18,8),  gridspec_kw={'wspace': 0.2})
    
    for i in range(0,6):
        
        time_df = time[(time['date'] < '2020-03-01')&( time['dayofweek'] == str(i))]
        time_df = time_df[~time_df['new_date'].isin(holiday)].reset_index(drop=True)    

        tmp = fp[(fp['city'] == city) & (fp['date'] >= '2020-01-20') & (fp['dayofweek'] == str(i))]
        tmp = tmp[~tmp['new_date'].isin(holiday)]
        group_= pd.DataFrame(tmp.groupby(['date'])['fp_num'].mean())
        group_.reset_index(inplace = True)

        join_data = pd.merge(time_df, group_, how = 'left', on = 'date')

        correlations =  join_data[['test','negative', 'confirmed', 'fp_num']].corr()


        # Heatmap of correlations
        fig.suptitle('{} correlations Every day of the week'.format(city), fontsize = 16)
        
        a = int(math.floor(i/2))
        b = i % 2
        fig.subplots_adjust(left=0.02, bottom=0.06, right=0.95, wspace=0.1)
        t= sns.heatmap(correlations, cmap = plt.cm.RdYlBu_r, vmin = -1, annot = True, vmax = 1, ax = ax[a,b])#, annot_kws={"size": 8})
        t.set_title("{}".format(dayofweek_map[i]), fontsize=8.5)
        
        
        correlations.reset_index(inplace = True)
        tmp = correlations.loc[0:2, 'fp_num']
        if i == 0:
            base = tmp.copy()
        else:

            base = pd.concat([base, tmp], axis = 1)


    base = base.T
    base.reset_index(drop= True, inplace = True)
    base.columns = ['test','negative','confirmed']
    week = pd.DataFrame(['Monday', 'Tuesday',  'Wednesday', 'Thursday', 'Friday', 'Saturday'], columns = ['week'])
    base = pd.concat([base,week], axis = 1)
    return base
    

def correlation_fp_num(base):
    plt.figure(figsize=(10, 8))
    for i in ['test','negative','confirmed']:
        plt.plot(base['week'], base[i])
        plt.legend()
    plt.ylabel('correlation between fp_num', fontsize = 15)
    plt.show()

## industrial Gu Correlation plot

In [ ]:
base = plot_correlatiin('Jongno-gu', time, fp)
correlation_fp_num(base)

In [ ]:
base = plot_correlatiin('Gangnam-gu', time, fp)
correlation_fp_num(base)

In [ ]:
base = plot_correlatiin('Jung-gu', time, fp)
correlation_fp_num(base)

In [ ]:
base = plot_correlatiin('Seocho-gu', time, fp)
correlation_fp_num(base)

### Great! On weekdays, Fp-num tends to decrease as COVID gets worse. <font size=3 color="red">(correlation is negative)</fond>
- It seems that there is a "social distance setting".
- Unfortunately, as the weekend approaches, such as Friday and Saturday, people begin to move little by little.
- Also There are some deviations on weekdays.


- 사회적 거리두기 운동이 이루어지고 있는 걸로 보입니다. 
- 안타깞게도 주말이 될수록 유동인구는 다소 증가하고 있습니다. 
- 또한 평일에는 약간의 편차가 존재합니다. 

## Residential Gu Correlation plot

In [ ]:
base = plot_correlatiin('Jungnang-gu', time, fp)
correlation_fp_num(base)

In [ ]:
base = plot_correlatiin('Nowon-gu', time, fp)
correlation_fp_num(base)



In [ ]:
base = plot_correlatiin('Gangbuk-gu', time, fp)
correlation_fp_num(base)



In [ ]:
base = plot_correlatiin('Gwanak-gu', time, fp)
correlation_fp_num(base)



### Great! As the COVID became more severe, the floating population increased in residential areas <font size=3 color="blue">(correlation is positive).</font>

- Unfortunately, the correlation decreases over the weekend, and it is estimated that people move to other industrial areas.


- 주거 지역이기 때문에 사람들의 COVID가 심해짐에 따라 유동인구와 양의 상관관계를 갖고 있습니다. 


# 2. For each age group Correlation plot(each city)
- This time, let's look at how the floating population changes as COVID increases with age.
- <font size=3 color="red">A negative correlation </font> : people are not moving as the corona situation becomes serious
- <font size=3 color="blue"> a positive correlation </font> : more and more people are coming to this area.


- 이번에는 같은 방식이지만, 각 연령대별로 나눠서 상관관계를 살펴보겠습니다.

In [ ]:
def birth_corr_plot(city, time, fp):

    holiday = ['1/01', '1/21','1/22','1/23','1/24','1/25','1/26','1/27','1/28','1/29','1/30']
    dayofweek_map = {0:'Monday', 1:'Tuesday', 2: 'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

    for j in [20, 30, 40, 50, 60, 70]:
        for i in range(0,6):

            time_df = time[(time['date'] < '2020-03-01')&( time['dayofweek'] == str(i))]
            time_df = time_df[~time_df['new_date'].isin(holiday)].reset_index(drop=True)    

            tmp = fp[(fp['city'] == city) & (fp['date'] >= '2020-01-20') & (fp['dayofweek'] == str(i)) & (fp['birth_year'] == j)]
            tmp = tmp[~tmp['new_date'].isin(holiday)]
            group_= pd.DataFrame(tmp.groupby(['date'])['fp_num'].mean())
            group_.reset_index(inplace = True)

            join_data = pd.merge(time_df, group_, how = 'left', on = 'date')

            correlations =  join_data[['test','negative', 'confirmed', 'fp_num']].corr()


            correlations.reset_index(inplace = True)
            tmp = correlations.loc[2:2, 'fp_num']
            if i == 0:
                base = tmp.copy()
            else:

                base = pd.concat([base, tmp], axis = 1)


        base = base.T
        base.reset_index(drop= True, inplace = True)
        base.columns = ['confirmed']
        week = pd.DataFrame(['Monday', 'Tuesday',  'Wednesday', 'Thursday', 'Friday', 'Saturday'], columns = ['week'])
        base = pd.concat([base,week], axis = 1)
        base['birth_year'] = j

        if j == 20:
            birth_base = base.copy()
        else:
            birth_base = pd.concat([birth_base, base], axis = 0)
            
    palette = sns.color_palette("mako_r", 6)
    plt.figure(figsize=(10, 8))
    sns.lineplot(data = birth_base, x = 'week', y='confirmed', hue='birth_year', palette=palette, sort=False)
    plt.show()







## industrial Gu Correlation plot

In [ ]:
birth_corr_plot('Jongno-gu', time, fp)


In [ ]:
birth_corr_plot('Gangnam-gu', time, fp)


In [ ]:
birth_corr_plot('Jung-gu', time, fp)


In [ ]:
birth_corr_plot('Seocho-gu', time, fp)



## Residential Gu Correlation plot

In [ ]:
birth_corr_plot('Jungnang-gu', time, fp)


In [ ]:
birth_corr_plot('Nowon-gu', time, fp)


In [ ]:
birth_corr_plot('Gangbuk-gu', time, fp)


In [ ]:
birth_corr_plot('Gwanak-gu', time, fp)


### The large flow is the same as before (whether the value is negative or positive), Variations vary by age group.
- For more detailed analysis, it is necessary to have data in the **dong unit** of the floating population, not in units of gu. 
- Or, it seems that We need more data about what event was in the gu.


- 큰 흐름은 동일하지만(값이 양이냐 음이냐) 연령대별로 편차가 심합니다. 
- 구 단위의 데이터가 아닌 동 단위의 유동인구 데이터가 제공되거나, 세부적으로 해당 구에 어떤 이벤트가 있었는지 파악이 필요할 것 같습니다.

# Conclusion

- In Korea, the "Social Distance Setting" is practiced very well in each region.
- Unfortunately, there seems to be some deviation when there is a lot of free time like weekends.
- Anyway, I think that this is one of the factors that has stabilized the trend of COVID infection in Korea.
- We hope that all countries in the world will regain their daily lives from COVID.
</br>


- 한국은 사회적 거리두기 운동을 자율적으로 지역별로 매우 잘 실천하고 있습니다. 
- 아쉽게도 주말처럼 여유 시간이 많을 때는 다소 이탈이 있습니다. 
- 아무튼 이러한 부분이 한국이 빠르게 COVID 감염 추세가 안정화된 요인 중에 하나라고 생각됩니다. 
- 세계의 모든 국가들이 COVID로부터 일상을 되찾기를 희망합니다.